## FEGROW


In [ ]:
import copy

import prody
from rdkit import Chem
from rdkit.Chem import AllChem

import fegrow
from fegrow import RGroups

import os

# For each protein


### attachments carried out

## tyk2

'ejm31_core_h' attach all to index 19. Five ligands grown, ['ejm44', 'ejm49', 'ejm51', 'ejm52', 'ejm53'].

## p38a
'2v_sub_h', attach index H4 - there are two Hs on the N so make sure correct is picked. Otherwise grows weirdly into ring. This is for 2cc and 2dd.
'2k_nosub', attach index H42. for 2q and 2u.

## mcl1
remove the following ligs from the r_groups : ['lig_39','lig_41','lig_51','lig_54','lig_55','lig_57','lig_62','lig_64','lig_66','lig_68']   
27_core_h, attach index 0, grows 'lig_29','lig_38','lig_40','lig_42','lig_44','lig_45'
60_sub_h, attach index 16, grows 'lig_23'
67_sub_h, attach index 16, grows 'lig_26'
from the newly grown ligands,
lig_42 is used as a template ligand, attach index X, grows lig_59


In [ ]:
# file locations, rename protein as needed
protein = "tyk2"
protein_folder = f"{protein}/fegrow"
protein_file = f"{protein}/protein/{protein}_parameterised.pdb"
r_group_folder = f"{protein_folder}/r_groups" # these are going to get made
smiles_file = f"{protein_folder}/r_groups/smiles.dat"
template_mol_folder = f"{protein_folder}/template_mol"

grown_mols_folder = f"{protein_folder}/final_noani"
pre_opt_folder = f"{protein_folder}/confomer_pre_optimisation"

# folders that need to be grown
folders = [grown_mols_folder, pre_opt_folder]
for folder in folders:
    if not os.path.isdir(folder):
        os.mkdir(folder)
        print(f"made dir {folder}")

In [ ]:
# want to make a list of dictionaries for the name, smiles, and attachment index
r_groups_dict_list = []

# open the smiles file
with open(smiles_file) as f:
    for line in f.readlines():
        smiles_string = line.split(',')[1]
        name = line.split(',')[0]

        # make from smiles
        smile = Chem.MolFromSmiles(f'{smiles_string}')
        # give molecule a name
        smile.SetProp("_Name",f"{name}")
        # add Hs to the molecule - this is important for adding 3D coordinates next
        smile_h = Chem.AddHs(smile)
        # add 3D coordinates by embedding the molecule, uses the ETKDG method
        # this is important so that not all the bonds are flat
        AllChem.EmbedMolecule(smile_h,
                                #randomSeed=0xf00d # this is for reproducibility
                                )

        # next want to dave this as a mol file, so that have and can also reopen for adding to rmols
        smile_mol = Chem.MolToMolBlock(smile_h)
        print(smile_mol,file=open(f'{r_group_folder}/{name}.mol','w+'))

# # adding r groups to a list
# selected_rgroups = []

# add your own R-groups files
for file in sorted(os.listdir(f"{r_group_folder}")):
    if file.endswith(".mol"):
        try:
            name = file.split(".")[0]
            r_group = Chem.MolFromMolFile(f"{r_group_folder}/{file}", removeHs=False)
            # append name to dictionary
            r_groups_dict_list.append({"name":f"{name}","r_group":r_group,"attach_idx":None, "template_mol":None})
        except:
            print(f"{file} could not be opened")


In [ ]:
# print specific r group if wanted
r_groups_dict_list[3]["r_group"]


In [ ]:
# remove any r groups
r_groups_remove = ['lig_39','lig_41','lig_51','lig_54','lig_55','lig_57','lig_62','lig_64','lig_66','lig_68']

for r_dict in r_groups_dict_list:
    if r_dict['name'] in r_groups_remove:
        r_groups_dict_list.remove(r_dict)

# r_groups_dict_list

In [ ]:
# !obabel coreh.pdb -O coreh.sdf -p 7

# make a dictionary
template_mol_dict = {}

# loading in the different templates, these are in sdf
for file in sorted(os.listdir(f"{template_mol_folder}")):
    if file.endswith(".sdf"):
        print(file)
        try:
            name = file.split(".")[0]
            # load in the template mols
            init_mol = Chem.SDMolSupplier(f'{template_mol_folder}/{file}', removeHs=False)[0]
            init_mol_h = Chem.AddHs(init_mol) # add H incase
            template_mol = fegrow.RMol(init_mol_h) # turn into fegrow template
            # append name to dictionary
            template_mol_dict[f"{name}"]=template_mol
        except:
            print(f"{file} could not be opened")

print(template_mol_dict)

In [ ]:
# visualise the different template mols
# can change based on name
template_mol_dict["ejm31_core_h"].rep2D(idx=True, size=(500, 500))

In [ ]:
# add the attachment index for each group. the length of the dictionary is:
print(f"no of indexes is {len(r_groups_dict_list)-1}, starting at 0. ({len(r_groups_dict_list)} in total)")

# for each index starting from 0, visualise the group
index_considering = 2
print(r_groups_dict_list[index_considering]["name"])
r_groups_dict_list[index_considering]["r_group"]

In [ ]:
# Then add the attachment index and mol for that group that was just visualised
r_groups_dict_list[index_considering]["template_mol"]="2k_nosub" # pick which
r_groups_dict_list[index_considering]["attach_idx"]=[42]


In [ ]:
# if want the same attachment index for all groups, use the below:
for r_group_dict in r_groups_dict_list:
    r_group_dict["attach_idx"]=[19]
    r_group_dict["template_mol"]="ejm31_core_h"

In [ ]:
# if want a certain attachment for list of ligands
# write list of ligs
attach_ligs = ['lig_29','lig_38','lig_40','lig_42','lig_44','lig_45']
# write for those in dict
for r_group_dict in r_groups_dict_list:
    if r_group_dict["name"] in attach_ligs:
        r_group_dict["attach_idx"]=[0]
        r_group_dict["template_mol"]="27_core_h"

In [ ]:
for r_dict in r_groups_dict_list:
    print(r_dict)

# if want to add to a molecule that needs to be grown first, the template_mol should be left empty
# check if all the entries are what they should be

grow the molecules:

In [ ]:
# dict of all the made fegrow molecules
rmols_single = {}

# list for r groups with no template mol to attach to yet
r_group_no_temp = []
# list for newly made molecules
r_group_new_mols = []

for r_group_dict in r_groups_dict_list:
  if not r_group_dict['template_mol']:
    r_group_no_temp.append(r_group_dict["name"])
  else:
    mol = fegrow.build_molecules(template_mol_dict[f"{r_group_dict['template_mol']}"], 
                                r_group_dict["attach_idx"], 
                                [r_group_dict["r_group"]])
    rmols_single[r_group_dict["name"]]=mol
    r_group_new_mols.append(r_group_dict["name"])

print(f"new mols made: {r_group_new_mols}")
print(f"no template yet: {r_group_no_temp}")

In [ ]:
# if want to add any of the no template yet to a newly made molecule:

# first want to add the new template molecule to the dict for it
# make list for new template mols
new_template_mols_name = ['lig_42']

for mol_name in new_template_mols_name:
    name = f"{mol_name}"
    template_mol = rmols_single[name]
    # append name to dictionary
    template_mol_dict[f"{name}"]=template_mol[0]

In [ ]:
print(template_mol_dict)
# visualise the new template mol so can pick attachment index
template_mol_dict["lig_42"].rep2D(idx=True, size=(500, 500))

In [ ]:
# add the template lig and attachment to the dict
# rerun this cell as needed to go through the new template mols added above
ligs_to_grow = ["lig_59"]

for r_group_dict in r_groups_dict_list:
    # if r_group_dict["name"] in r_group_no_temp:
    if r_group_dict["name"] in ligs_to_grow:
        r_group_dict["attach_idx"]=[22]
        r_group_dict["template_mol"]="lig_42"

In [ ]:
# run this cell, which is a repeat of the previous one
# it will make a new list for any molecules that are still missing
# the rmols single will be updated 

# list for r groups with no template mol to attach to yet
r_group_no_temp = []
# list for newly made molecules
r_group_new_mols = []

for r_group_dict in r_groups_dict_list:
  if not r_group_dict['template_mol']:
    r_group_no_temp.append(r_group_dict["name"])
  else:
    print(r_group_dict["name"])
    mol = fegrow.build_molecules(template_mol_dict[f"{r_group_dict['template_mol']}"], 
                                r_group_dict["attach_idx"], 
                                [r_group_dict["r_group"]])
    rmols_single[r_group_dict["name"]]=mol
    r_group_new_mols.append(r_group_dict["name"])

print(f"new mols made: {r_group_new_mols}")
print(f"no template yet: {r_group_no_temp}")

# if there is still no template yet, repeat the above cells

In [ ]:
# sorted list of all the made molecules
made_rmols = []

# combine into single rmols file
rmols = fegrow.RList()
id_counter = 0
for key in rmols_single.keys():
  mol = rmols_single[key][0]
  mol.id = id_counter
  rmols.append(mol)
  made_rmols.append(key)
  id_counter += 1

print(made_rmols)
rmols
# rmols.rep2D()

In [ ]:
# visualise if needed
rmols[0].rep3D()

For each ligand, a specified number of conformers (`num_conf`) is generated by using the RDKit [ETKDG algorithm](https://doi.org/10.1021/acs.jcim.5b00654). Conformers that are too similar to an existing structure are discarded. Empirically, we have found that `num_conf=200` gives an exhaustive search, and `num_conf=50` gives a reasonable, fast search, in most cases.

If required, a third argument can be added `flexible=[0,1,...]`, which provides a list of additional atoms in the core that are allowed to be flexible. This is useful, for example, if growing from a methyl group and you would like the added R-group to freely rotate.

In [ ]:
rmols.generate_conformers(num_conf=50, 
                          minimum_conf_rms=0.5, 
                          # flexible=[12,13,14]
                        )

In [ ]:
# load back into prody
rec_final = prody.parsePDB(f"{protein_file}")

In [ ]:
rmols[2].rep2D()

In [ ]:
# can visualise the diff conformers in the protein
rmols[2].rep3D(prody=rec_final)

In [ ]:
# remove any clashing confomers
rmols.remove_clashing_confs(rec_final)
# does this for all rmols indexes, not just the one visualised.

In [ ]:
# can visualise again
rmols[1].rep3D(prody=rec_final)

In [ ]:
# save the confomers for comparison to optimise in receptor

with Chem.SDWriter(f'{pre_opt_folder}/best_confomer_each.sdf') as w:
    for m, name in zip(rmols, made_rmols):
        m.SetProp("_Name",f"{name}")
        w.write(m)


### Optimise conformers in context of protein

The remaining conformers are optimised using hybrid machine learning / molecular mechanics (ML/MM), using the [ANI2x](https://doi.org/10.1021/acs.jctc.0c00121) neural nework potential for the ligand energetics (as long as it contains only the atoms H, C, N, O, F, S, Cl). Note that the Open Force Field [Parsley](https://doi.org/10.1021/acs.jctc.1c00571) force field is used for intermolecular interactions with the receptor.

`sigma_scale_factor`: is used to scale the Lennard-Jones radii of the atoms.

`relative_permittivity`: is used to scale the electrostatic interactions with the protein.

`water_model`: can be used to set the force field for any water molecules present in the binding site.

In [ ]:
# opt_mol, energies
energies = rmols.optimise_in_receptor(
    receptor_file=protein_file, 
    ligand_force_field="gaff", 
    use_ani=False,
    sigma_scale_factor=0.8,
    relative_permittivity=4,
    water_model = "tip3p.xml"
)

Any of the rmols that have no available conformers (due to unresolvable steric clashes with the protein) can be discarded using the `.discard_missing()` function. This function also returns a list of the indices that were removed, which can be helpful when carrying out data analysis.

In [ ]:
missing_ids = rmols.discard_missing()

Optionally, display the final optimised conformers. Note that, unlike classical force fields, ANI allows bond breaking. You may occasionally see ligands with distorted structures and very long bonds, but in our experience these are rarely amongst the low energy structures and can be ignored.

In [ ]:
rmols[4].rep3D(prody=rec_final)

Conformers are now sorted by energy, only retaining those within 5 kcal/mol of the lowest energy structure:

In [ ]:
final_energies = rmols.sort_conformers(energy_range=5)

Save all of the lowest energy conformers to files and print the sorted energies in kcal/mol (shifted so that the lowest energy conformer is zero).

In [ ]:
# save the confomers for comparison to optimise in receptor

with Chem.SDWriter(f'{grown_mols_folder}/best_confomer_each.sdf') as w:
    for m, name in zip(rmols, made_rmols):
        m.SetProp("_Name",f"{name}")
        w.write(m)


In [ ]:
# for wanting to seperate the written sdf files

file = f"{grown_mols_folder}/best_confomer_each.sdf"

#get all the ligand names from the file
ligand_names = []

with open(file, "r") as f:
    for line in f.readlines():
        if "lig_" in line:
            name =f"{line.split(' ')[0].strip()}"
            ligand_names.append(name)

# for lig in ligand_names:
#     if "-" in lig:
#         ligand_names.remove(lig)

print(ligand_names)


In [ ]:
# load all molecules
mols_file = Chem.SDMolSupplier(file, removeHs=False)

count = 0

# write all of the best confomers to their own file
for mol in mols_file:
    name = ligand_names[count]
    # mol_h = Chem.AddHs(mol) # if need to add H
    writer = Chem.rdmolfiles.SDWriter(f"{grown_mols_folder}/{name}.sdf")
    writer.write(mol) # need to change this to mol_h then
    count += 1

In [ ]:
# check if all names and files length match
print(len(ligand_names))
print(len(mols_file))

In [ ]:
print(final_energies)

In [ ]:
# fix H for single files
# load all molecules
folder_path = f"{protein}/fegrow/final"

name = "2u"

file = f"{folder_path}/{name}.sdf"
mol = Chem.SDMolSupplier(file, removeHs=False)[0]
print(mol.GetNumAtoms())
mol_h = Chem.AddHs(mol)
print(mol_h.GetNumAtoms())
# AllChem.EmbedMolecule(mol_h,
#                         #randomSeed=0xf00d # this is for reproducibility
#                         )
writer = Chem.rdmolfiles.SDWriter(f"{protein}/ligands/{name}.sdf")
writer.write(mol_h)

In [ ]:
## from old script

# save the confomers and pdb and sdf
[rmol.to_file(f"{confomers_dir}/best_conformers_{i}.pdb") for i, rmol in enumerate(rmols)]
[rmol.to_file(f"{confomers_dir}/best_conformers_{i}.sdf") for i, rmol in enumerate(rmols)]

# write the first of each so best conformation
with Chem.SDWriter(f'{confomers_dir}/besteach.sdf') as w:
    for m in rmols:
        w.write(m)

# rename to the ligand name and only save the best confomer
pdb_ligand_files = sorted(glob.glob(f"{confomers_dir}/best_conformers*.pdb"))
sdf_ligand_files = sorted(glob.glob(f"{confomers_dir}/best_conformers*.sdf"))

files_ext = [pdb_ligand_files, sdf_ligand_files]
extension = ["pdb","sdf"]

for files, ext in zip(files_ext, extension):
    for file, name in zip(files, r_group_names):
        with open(file,"r") as input:
            with open(f"{path_to_ligands}/{name}.{ext}","w") as output:
                for line in input:
                    output.write(line)
                    if "END" in line:
                        if ext == "sdf":
                            output.write("$$$$")
                        break